In [ ]:
!pip install idx2numpy

In [ ]:
import numpy as np

from src.preprocessing import *
from src.evaluation import accuracy
from src.multiclass_model import LogisticRegressionMulti
from src.hyperparameters_tuning import RandomSearch

In [ ]:
def is_composite(y: np.ndarray) -> np.ndarray:
    return np.where(np.isin(y, [4,6,8,9]), 1,-1)

In [ ]:
!mkdir data
!wget -P data/ http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz;
!wget -P data/ http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz;
!wget -P data/ http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz;
!wget -P data/ http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz;
!gunzip data/*

## Load data

In [ ]:
X_train, y_train = load_data("train")
X_test, y_test = load_data("t10k")

split = X_train.shape[0]

X = np.vstack((X_train,X_test))

X = X.reshape(-1,28*28)

# Przeskalowanie danych
scaler = Scaler()
scaler.fit(X)
X_scaled = scaler.standardize(X)

X_pca = PCA(X_scaled, 150)
X_pca_poly = Polynomial(X_pca)

In [105]:
X_pca_poly.shape

(55220, 11325)

In [ ]:
X_train_pca_poly = X_pca_poly[:split,:]
X_test_pca_poly = X_pca_poly[split:,:]

In [108]:
X_train_pca_poly.shape, y_train.shape

((47335, 11325), (47335,))

## Random Search

In [ ]:
params = {"C" : [1, 2, 5, 10],
          "learning_rate_init" : [10**(-x) for x in np.arange(1, 7, 1, dtype="float")],
          "weight_decay" : np.arange(5, 10, 1) * 0.1,
          "alpha" : np.arange(5, 9, 1) * 0.1,
          "val_split": [1]
}

In [ ]:
rs = RandomSearch(parameters_dict=params, n_samples=10, validation_split=0.9)

In [33]:
rs.fit(LogisticRegressionMulti, X_pca_poly, y, n_model_iter=8)

(0.957752429235319,
 {'C': 5,
  'alpha': 0.5,
  'learning_rate_init': 0.1,
  'val_split': 1,
  'weight_decay': 0.6000000000000001})

## Train on whole data

In [ ]:
# Dodałem większe C, dlatego że w zbiór testowy składa się z dwóch części :
# jest jest czystsza i latwiesza do rozpoznawania, a druga pochodzi z innego zbioru i jest "brudniejsza"  

In [ ]:
model = LogisticRegressionMulti(C=10, learning_rate_init=0.1, alpha=0.5, weight_decay=0.6,
                                max_iter=15, batch_size=32, val_split = 1, verbose=True,         
                                num_classes=8, random_state=25)

In [ ]:
model.fit(X_train_pca_poly, y_train.reshape(-1,1));

In [112]:
accuracy(model, X_test_pca_poly, y_test)

0.954724159797083

In [116]:
#sprawdzimy czy istnieje różnica w danych testowych?
accuracy(model, X_test_pca_poly[5000:], y_test[5000:]), accuracy(model, X_test_pca_poly[:5000], y_test[:5000]) # Tak

(0.9681109185441941, 0.947)

In [ ]:
predict_comp = preds_comp = np.where(np.isin(model.predict(X_test_pca_poly), [4,6,8,9]), 1,-1)
y_test_comp = is_composite(y_test)
acc_comp = (sum(predict_comp == y_test_comp)/ len(y_test_comp))

In [114]:
acc_comp

0.97285986049461

In [ ]:
model.save_model("logreg_multiclass_trained.json")